In [3]:
from utils import (
    wn, 
    get_normal_form, 
    my_split, 
    TitleInWn,
    is_lat
)
import json
from os import listdir
from os.path import isfile, join
from config.const import DAMP_OF_WIKIDATA_PATH, GOLD_WIKIDATA_DATASET, ALL_NEED_ARTICLE
from collections import defaultdict
import pickle
import pandas as pd
from tqdm import tqdm
from collections import deque
from xml.dom import minidom
from dataclasses import dataclass

In [ ]:
set_senses = set([' '.join([get_normal_form(w).lower() for w in s.lemma.split()]) for s in wn.senses])

In [ ]:
onlyfiles = [f for f in listdir(DAMP_OF_WIKIDATA_PATH) if isfile(join(DAMP_OF_WIKIDATA_PATH, f))]

In [4]:
count  = 0
onlyfiles = [f for f in listdir(DAMP_OF_WIKIDATA_PATH) if isfile(join(DAMP_OF_WIKIDATA_PATH, f))]
for file in onlyfiles:
    with open(f'{DAMP_OF_WIKIDATA_PATH}\\{file}', 'r', encoding='utf-8') as f:
        for _, line in enumerate(f):
            page = json.loads(line)
            if (page['id'] and int(page['id'][1:]) > 10000000) or 'ru' in page['descriptions']:
                print(page)
                count += 1
            if count  >= 10:
                break           
        if count  >= 10:
            break

31 Q31
{'id': 'Q31', 'pageid': 127, 'label': {'ru': 'Бельгия', 'en': 'Belgium'}, 'descriptions': {'ru': 'государство в Западной Европе', 'en': 'country in western Europe'}, 'aliases': {'en': ['Kingdom of Belgium', 'BEL', 'be', '🇧🇪', 'BE', 'Koninkrijk België', 'Royaume de Belgique', 'Königreich Belgien', 'Belgien'], 'ru': ['Belgique', 'Королевство Бельгия']}, 'sitelinks': {'enwiki': 'Belgium', 'ruwiki': 'Бельгия'}, 'rels': [{'rel_id': 'Q3624078', 'rank': 'preferred', 'type': 'instance_of'}, {'rel_id': 'Q43702', 'rank': 'normal', 'type': 'instance_of'}, {'rel_id': 'Q6256', 'rank': 'preferred', 'type': 'instance_of'}, {'rel_id': 'Q20181813', 'rank': 'normal', 'type': 'instance_of'}, {'rel_id': 'Q185441', 'rank': 'normal', 'type': 'instance_of'}, {'rel_id': 'Q1250464', 'rank': 'normal', 'type': 'instance_of'}, {'rel_id': 'Q113489728', 'rank': 'normal', 'type': 'instance_of'}]}
8 Q8
{'id': 'Q8', 'pageid': 134, 'label': {'en': 'happiness', 'ru': 'счастье'}, 'descriptions': {'en': 'mental or 

In [ ]:
# with open("D:\\lbase_data\\all_straight_graph.pickle", 'wb') as f:
#     pickle.dump(straight_idx,f)
# with open("D:\\lbase_data\\all_inverse_graph.pickle", 'wb') as f:
#     pickle.dump(inverse_idx,f)

In [ ]:
# with open("D:\\lbase_data\\all_straight_graph.pickle", 'rb') as f:
#     straight_idx = pickle.load(f)
# with open("D:\\lbase_data\\all_inverse_graph.pickle", 'rb') as f:
#     inverse_idx = pickle.load(f)

In [ ]:
# with open("D:\\lbase_data\\clear_inverse_graph.pickle", 'wb') as f:
#     pickle.dump(clear_inverse_idx,f)
# with open("D:\\lbase_data\\clear_inverse_graph.pickle", 'rb') as f:
#     clear_inverse_idx = pickle.load(f)

In [ ]:
tweets = []
to_add = set()
for file in tqdm(onlyfiles):
    with open(f'{DAMP_OF_WIKIDATA_PATh}\\{file}', 'r', encoding='utf-8') as f:
        for line in f:
            info = json.loads(line)
            if info['id'] == 'Q190':
                    print(info)
            if 'ru' in info['label'] and int(info['id'][1:]) <= 10000000 and (is_lat(info["label"]['ru'])) and 'фильм' not in info["label"]['ru']:
                try:
                    label = info["label"]['ru']
                    sense = wn.get_senses(label)
                    if info['id'] == 'Q190':
                            print(sense)        
                    if sense:
                        tweets.append((info, sense[0].lemma))
                        for elem in info['rels']:
                            to_add.add(elem['rel_id'])
                    else:
                        lemma = TitleInWn(set_senses, my_split(label).split(',')[0])
                        if info['id'] == 'Q190':
                            print(lemma, '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!') 
                        if lemma:
                            tweets.append((info, lemma))
                            for elem in info['rels']:
                                to_add.add(elem['rel_id'])     
                except:
                    pass          

In [ ]:
len(tweets)

In [ ]:
len(to_add)

In [ ]:
for i in tqdm(range(5)):
    if i != 0:
        to_add = new_to_add
    print(len(to_add))
    new_to_add = set()
    for file in onlyfiles:
        with open(f'{DAMP_OF_WIKIDATA_PATh}\\{file}', 'r', encoding='utf-8') as f:
            for line in f:
                info = json.loads(line)
                if info['id'] in to_add and int(info['id'][1:]) <= 10000000 and 'ru' in info['label']  and 'фильм' not in info["label"]['ru']:
                    try:
                        label = info["label"]['ru']
                        sense = wn.get_senses(label)
                        if sense:
                            tweets.append((info, sense[0].lemma))
                        else:
                            lemma = TitleInWn(set_senses, my_split(label).split(',')[0])
                            if lemma is not None:
                                tweets.append((info, lemma))
                        for elem in info['rels']:
                            new_to_add.add(elem['rel_id'])
                    except:
                        pass
                elif info['label'] and info['id'] in to_add:
                    for elem in info['rels']:
                        new_to_add.add(elem['rel_id'])
                    tweets.append((info, info['label']['en']))

In [ ]:
len(tweets)

In [ ]:
# with open(ALL_NEED_ARTICLE, 'wb') as f:
#     pickle.dump(tweets, f)

In [ ]:
data = pd.read_csv(GOLD_WIKIDATA_DATASET)
data

In [ ]:
with open(ALL_NEED_ARTICLE, 'rb') as f:
    articles = pickle.load(f)
len(articles)

In [ ]:
graph_path_straight = defaultdict(set)
graph_path_inverse = defaultdict(set)
id_artircle2idx_article = {}
idx_article2id_artircle = {}
for idx, (article, _) in tqdm(enumerate(articles)):
    id_artircle2idx_article[article['id']] = idx
    idx_article2id_artircle[idx] = article['id']
    for link in article['rels']:
        graph_path_straight[link['rel_id']].add(article['id'])
        graph_path_inverse[article['id']].add(link['rel_id'])

In [ ]:
def find_reachable_vertices(graph, start_vertex):
    visited = set()
    queue = deque([start_vertex])
    reachable_vertices = set()

    while queue:
        current_vertex = queue.popleft()
        visited.add(current_vertex)
        reachable_vertices.add(current_vertex)

        for neighbor in graph[current_vertex]:
            if neighbor not in visited:
                queue.append(neighbor)

    return reachable_vertices

In [ ]:
print(find_reachable_vertices(graph_path_inverse, 'Q4568'))

In [ ]:
def extractCtxS(wn, lemma:str):
    #составляем контекст для слова из wordnet
    ctx_s = set()
    #synonymy
    for sense in wn.get_synsets(lemma):
        for synonymy in sense.senses:
            ctx_s.update(my_split(synonymy.lemma).split(","))
    #Hypernymy/Hyponymy
    for sense in wn.get_senses(lemma):
        for hypernyms in sense.synset.hypernyms:
            ctx_s.update(my_split(hypernyms.title).split(","))
    for sense in wn.get_senses(lemma):
        for hyponyms in sense.synset.hyponyms:
            ctx_s.update(my_split(hyponyms.title).split(","))
    #Sisterhood:
    for sense in wn.get_senses(lemma):
        for hypernyms in sense.synset.hypernyms:
            for sister in hypernyms.hyponyms:
                ctx_s.update(my_split(sister.title).split(","))
    return ctx_s
@dataclass
class WnCtx:
    id: int
    ctx: set
    lemmaInWn: str
    name: str

In [ ]:
file = open("D:\\asd\\ctxS.txt", "rb")
unpickler = pickle.Unpickler(file)
dictWn = unpickler.load()
file.close()

In [ ]:
wn.get_synsets(wn['130555-N-741490'].lemma), wn['130555-N-741490']

In [ ]:
from typing import Tuple, List

In [ ]:
def get_new_lemma(all_synsets, title):
    title = title.lower()
    title = title.replace("—", "-")
    title = title.replace(",", "")
    if title in all_synsets:
        return title
    if "(" in title:
        text = my_split(title).split(",")
        if text[0] in all_synsets:
            return text[0]
    text = title.split(",")
    lemmatized = " ".join([get_normal_form(word).lower()
                for word in text])
    if lemmatized in all_synsets:
        return lemmatized
    if "ё" in title:
        return get_new_lemma(all_synsets, title.replace("ё","е"))
    return None

In [ ]:
all_synsets = set([lem.title.lower() for lem in wn.synsets])

In [ ]:

def create_dict_candidates(articles:List[Tuple[dict, str]]) -> dict:
    """
        Создание словаря из синсета в кандидаты для связывания
    """
    canidates = defaultdict(list)
    for (article, lemma) in tqdm(articles):
        if 'ru' in article['label']:
            if lemma:
                synset_base_on_lemma = wn.get_synsets(lemma)
                if synset_base_on_lemma:
                    for synset in wn.get_synsets(lemma):
                        canidates[synset.id].append((article, lemma))
            else:
                lemma_new = get_new_lemma(all_synsets, article['label']['ru'])
                if lemma_new:
                    for synset in wn.get_synsets(lemma_new):
                        canidates[synset.id].append((article, lemma_new))
    return canidates
candidates = create_dict_candidates(articles)
print(len(candidates))

In [ ]:
# candidates['ЭНЦИКЛОПЕДИЧЕСКИЙ СЛОВАРЬ']
import re


def clean_text(text):
    cleaned_text = re.sub(r"[^\w\s]", "", text)
    cleaned_text = re.sub(r"\d+", "", cleaned_text)
    cleaned_text = re.sub(r"\s+", " ", cleaned_text)
    cleaned_text = cleaned_text.strip().lower()
    return cleaned_text

def extract_ctx_wikidata(article:dict) -> set:
    '''
        Излвечения контекста статьи из ее ооописания на русском
    '''
    ctx = set()
    if 'ru' in article['descriptions']:
        for token in clean_text(article['descriptions']['ru']).split():
            ctx.add(get_normal_form(token).lower())
    return ctx
def get_score(elem1:set, elem2:set) -> float:
    '''
        Функция подсчета близости статьи и контекстов на основе эвристики
    '''
    return (len((elem1 & elem2)) + 1) / (len(elem1) + len(elem2) + 1)

In [ ]:
score_dict = defaultdict(list)
for i, (title_synset, candidatess) in tqdm(enumerate(candidates.items())):
    for candidate, lemma in candidatess:
        if 'N' in wn[lemma][0].id:
            synset_ctx = dictWn[wn[lemma][0].id].ctx
            article_ctx = extract_ctx_wikidata(candidate)
            article_ctx.update([candidate['label']['ru'].lower(), lemma.lower()])
            score = get_score(article_ctx, synset_ctx)
            score_dict[title_synset].append(score)
        else:
            score_dict[title_synset].append(0.0)

In [ ]:
for candidate, lemma in candidates['103410-N']:
        if 'N' in wn[lemma][0].id:
            synset_ctx = dictWn[wn[lemma][0].id].ctx
            article_ctx = extract_ctx_wikidata(candidate)
            article_ctx.update([candidate['label']['ru'], lemma])
            score = get_score(article_ctx, synset_ctx)
            print(candidate['id'],article_ctx)
            score_dict[title_synset].append(score)
        else:
            score_dict[title_synset].append(0.0)

In [ ]:
@dataclass
class display_synset_to_wikidata:
    id:int
    label:int
    lemma:str
    sense_id:int
    score:float
    synset_lemma:str

In [ ]:
res_score={}
for _, (title_synset, candidatess) in tqdm(enumerate(candidates.items())):
    sorted_lst = sorted(zip(candidatess, score_dict[title_synset]), key=lambda x: x[1], reverse=True)
    if sorted_lst and 'N' in title_synset:
        res_score[title_synset] = display_synset_to_wikidata(sorted_lst[0][0][0]['id'], sorted_lst[0][0][0]['label'], sorted_lst[0][0][1], wn[sorted_lst[0][0][1]][0].id, sorted_lst[0][1], title_synset)

In [ ]:
# with open("res.pickle", 'wb') as f:
#     pickle.dump(res_score, f)

In [ ]:
with open("res.pickle", 'rb') as f:
    res_score = pickle.load(f)

In [ ]:
len(res_score)

In [ ]:

data['predict_id'] = data['synset_id'].apply(lambda x: res_score[x].id if x in res_score else 'не связан')
data


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(data[data['predict_id'] !='не связан'].predict_id.astype(str), data[data['predict_id'] !='не связан'].WikiDataGoldId.astype(str)), data[data['predict_id'] !='не связан'].shape

In [ ]:
sum([1 if i in candidates else 0 for i in data['synset_id'].values]), wn['наставничество'][0].lemma, wn.get_synsets('НАСТАВНИЧЕСТВО')

In [ ]:
id_artircle2idx_article['Q7209862']

In [ ]:
articles[50899]

In [ ]:
candidates['124807-N']

In [2]:
pip show fasttext

Name: fasttext
Version: 0.9.2
Summary: fasttext Python bindings
Home-page: https://github.com/facebookresearch/fastText
Author: Onur Celebi
Author-email: celebio@fb.com
License: MIT
Location: c:\users\professional\anaconda3\lib\site-packages
Requires: numpy, pybind11, setuptools
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [10]:
import fasttext
ft = fasttext.load_model('E:\\data_diplom\\cc.ru.300.bin')

In [13]:
from src.mapWikipedia import SentenceBertTransformer,cosine_similarity
labse = SentenceBertTransformer( device="cuda")
labse.load_model()
print(labse.cosine_similarity('Как дела?', 'Как дела у тебя'))


0.8650397


In [14]:
cosine_similarity(ft.get_word_vector('Как дела?'), ft.get_word_vector('Как дела у тебя'))

0.6573649

In [11]:
ft.get_word_vector('слово').shape

(300,)

In [12]:
ft.get_nearest_neighbors('слово')

[(0.8006083369255066, 'словосочетание'),
 (0.7432404160499573, 'словечко'),
 (0.7346797585487366, 'слово-'),
 (0.7283103466033936, 'Слово'),
 (0.7168722748756409, 'слово.'),
 (0.6934490203857422, '-слово'),
 (0.6674397587776184, 'Cлово'),
 (0.6530631184577942, '.Слово'),
 (0.6497068405151367, 'слово.-'),
 (0.6411762237548828, 'ругательство')]

In [ ]:
from src.mapWikipedia import read_dump, create_wikisynset,create_info_about_sense, wn, unambiguous_bindings, read_pkl
from config.const import PATH_TO_TMP_FILE
map = read_pkl(PATH_TO_TMP_FILE+'fst.pkl')

In [1]:
from src.mapWikipedia import read_dump, delete_double_in_candidates,create_info_about_sense,read_pkl, wn, second_stage_bindings,create_candidates_index_dict, wn, create_candidates_for_multi_stage,multi_bindings_stage
from src.mapWikidata import extract_title_with_title_in_RuWordNet
from config.const import DAMP_OF_WIKIPEDIA_PATH, GOLD_DATA,PATH_TO_TMP_FILE
# pages, dictPageRedirect, dictRedirect =read_dump(DAMP_OF_WIKIPEDIA_PATH)
# dictWn = create_info_about_sense()
# wiki = create_wikisynset(pages, dictPageRedirect)
# get_sense_id_by_title('Выставка')
# k, n = unambiguous_bindings(wn, dictWn, wiki, mode='read')

c:\Users\Professional\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [35]:
c, r = extract_title_with_title_in_RuWordNet(mode='read')


Start reading from file
Successful recording


WikidataPage(page={'id': 'Q31', 'pageid': 127, 'label': {'ru': 'Бельгия', 'en': 'Belgium'}, 'descriptions': {'ru': 'государство в Западной Европе', 'en': 'country in western Europe'}, 'aliases': {'en': ['Kingdom of Belgium', 'BEL', 'be', '🇧🇪', 'BE', 'Koninkrijk België', 'Royaume de Belgique', 'Königreich Belgien', 'Belgien'], 'ru': ['Belgique', 'Королевство Бельгия']}, 'sitelinks': {'enwiki': 'Belgium', 'ruwiki': 'Бельгия'}, 'rels': [{'rel_id': 'Q3624078', 'rank': 'preferred', 'type': 'instance_of'}, {'rel_id': 'Q43702', 'rank': 'normal', 'type': 'instance_of'}, {'rel_id': 'Q6256', 'rank': 'preferred', 'type': 'instance_of'}, {'rel_id': 'Q20181813', 'rank': 'normal', 'type': 'instance_of'}, {'rel_id': 'Q185441', 'rank': 'normal', 'type': 'instance_of'}, {'rel_id': 'Q1250464', 'rank': 'normal', 'type': 'instance_of'}, {'rel_id': 'Q113489728', 'rank': 'normal', 'type': 'instance_of'}]}, lemma='БЕЛЬГИЯ')

In [4]:
dictWn = create_info_about_sense()

dictt, new_wiki = second_stage_bindings()

dictLemmNew = create_candidates_index_dict(name='lst_candidates_after_snd_stage.pkl', mode='read')


dict_candidtes = create_candidates_for_multi_stage(new_wiki, wn, dictWn, dictLemmNew, mode='read')

dict_candidtes_update = delete_double_in_candidates(dict_candidtes)

Start reading from file
Successful reading
Start reading from file
Successful reading
Start reading from file
Successful reading
Start reading from file = E:\data_diplom\candidates_for_multi_stage.pkl
Successful reading
Strt deleted doubles


100%|██████████| 20327/20327 [00:01<00:00, 12233.84it/s]

Was deleted 293326 doubles


In [5]:
dicttFinal_labse, sort_candidates_labse = multi_bindings_stage(type_bindings='labse', model_name='setu4993/LaBSE', mode='read')

_labsesetu4993LaBSE
Start reading from file: E:\data_diplom\thr_dict__labsesetu4993LaBSE.pkl


FileNotFoundError: [Errno 2] No such file or directory: 'E:\\data_diplom\\thr_dict__labsesetu4993LaBSE.pkl'

{'1-N': [(WikiSynset(), 0.3276652972449847),
  (WikiSynset(), 0.21316744332891752),
  (WikiSynset(), 0.2595583527604292),
  (WikiSynset(), 0.45030686235805306)],
 '10-N': [(WikiSynset(), 0.49027892632756054),
  (WikiSynset(), 0.4209187732009866),
  (WikiSynset(), 0.3187197986368391),
  (WikiSynset(), 0.3638027079924489)],
 '100-N': [(WikiSynset(), 1)],
 '100006-N': [(WikiSynset(), 0.4879999200151201),
  (WikiSynset(), 0.2719660643945313)],
 '100007-N': [(WikiSynset(), 0.4923798755310173),
  (WikiSynset(), 0.47499739941690616),
  (WikiSynset(), 0.3832547459534334),
  (WikiSynset(), 0.44670175861040967),
  (WikiSynset(), 0.46296786932864925),
  (WikiSynset(), 0.3918740074920005),
  (WikiSynset(), 0.4526293128874533),
  (WikiSynset(), 0.34282957767827116),
  (WikiSynset(), 0.4661768278290001),
  (WikiSynset(), 0.4838533006635144),
  (WikiSynset(), 0.29763943832780965)],
 '100008-N': [(WikiSynset(), 0.5376899256646736),
  (WikiSynset(), 0.5067559541194697),
  (WikiSynset(), 0.5092084349250

In [24]:
mappp = read_pkl(path=PATH_TO_TMP_FILE+'thr_dict__fasttext.pkl')

In [25]:
# mappp, sorted = dicttFinal[0], dicttFinal[1]
len(mappp)

21697

In [26]:
import pickle
file = open("E:\\data_diplom\\snd_dict.pkl", "rb")
unpickler = pickle.Unpickler(file)
single = unpickler.load()
file.close()
set_single = []
for key, value in single.items():
    set_single.append(value.wordId)
set_single = set(set_single)

In [6]:
import pandas as pd
from sklearn.metrics import accuracy_score
df = pd.read_csv(GOLD_DATA)
synset_id = set(df["synset_id"].values)
df

,Unnamed: 0,synset_id,wiki_title,annotation,wiki_title_gold
0,0,115195-N,Облава,Да,Облава
1,1,128158-N,Шаньдун,Да,Шаньдун
2,2,143699-N,Оксюморон,Да,Оксюморон
3,3,4061-N,Доверенность,Да,Доверенность
4,5,136544-N,Шахматная доска,Да,Шахматная доска
...,...,...,...,...,...
1725,1993,113701-N,Царю небесный,Нет,Бог
1726,1994,124807-N,Наставничество,Да,Наставничество
1727,1995,9199-N,Трансурановые элементы,Да,Трансурановые элементы
1728,1996,103410-N,Эсватини,Да,Эсватини


In [28]:
dict_for_check = {}
for key, value in mappp.items():
    if value.wordId in synset_id:
        dict_for_check[value.wordId] = value.title
dataset = df[["synset_id", "wiki_title_gold"]]
for_apply = df["synset_id"]
dataset["wiki_title"] = for_apply.apply(lambda x: dict_for_check[x] if x in dict_for_check else "не связан")
dataset

C:\Users\Professional\AppData\Local\Temp\ipykernel_3748\3409875238.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["wiki_title"] = for_apply.apply(lambda x: dict_for_check[x] if x in dict_for_check else "не связан")


,synset_id,wiki_title_gold,wiki_title
0,115195-N,Облава,Метод Крекера (телесериал)
1,128158-N,Шаньдун,Шаньдун (авианосец)
2,143699-N,Оксюморон,Оксюморон
3,4061-N,Доверенность,Доверенность
4,136544-N,Шахматная доска,Шахматная доска
...,...,...,...
1725,113701-N,Бог,Бог (Южный Парк)
1726,124807-N,Наставничество,Ментор (спутник)
1727,9199-N,Трансурановые элементы,Трансурановые элементы
1728,103410-N,Эсватини,Эсватини


In [29]:

y_pred = dataset["wiki_title"].values
y_true = dataset["wiki_title_gold"].values
accuracy_score(y_true, y_pred)

0.6063583815028901

In [30]:
dataset[dataset["wiki_title"]=="не связан"].shape

(73, 3)

In [31]:
for_apply = df["synset_id"].copy()
dataset["multi"] = for_apply.apply(lambda x: 0 if x in set_single else 1)
dataset

C:\Users\Professional\AppData\Local\Temp\ipykernel_3748\183366674.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["multi"] = for_apply.apply(lambda x: 0 if x in set_single else 1)


,synset_id,wiki_title_gold,wiki_title,multi
0,115195-N,Облава,Метод Крекера (телесериал),1
1,128158-N,Шаньдун,Шаньдун (авианосец),1
2,143699-N,Оксюморон,Оксюморон,0
3,4061-N,Доверенность,Доверенность,1
4,136544-N,Шахматная доска,Шахматная доска,1
...,...,...,...,...
1725,113701-N,Бог,Бог (Южный Парк),0
1726,124807-N,Наставничество,Ментор (спутник),0
1727,9199-N,Трансурановые элементы,Трансурановые элементы,1
1728,103410-N,Эсватини,Эсватини,0


In [32]:
y_true = dataset.loc[dataset['multi'] == 0, 'wiki_title_gold'].values
y_pred = dataset.loc[dataset['multi'] == 0, 'wiki_title'].values
accuracy_score(y_true, y_pred)

0.6826347305389222

In [33]:
y_true = dataset.loc[dataset['multi'] == 1, 'wiki_title_gold'].values
y_pred = dataset.loc[dataset['multi'] == 1, 'wiki_title'].values
accuracy_score(y_true, y_pred)

0.5583804143126178